In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from fastdtw import fastdtw
from tqdm import tqdm
import tslearn
import tslearn.metrics
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='ignore', category=CollinearityWarning)
import warnings

from perform_utils import *
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import seaborn as sns

cmap_blue = cm.get_cmap('Blues')
cmap_red = cm.get_cmap('Reds')

def beta_calculation(y,x):
    
    x = x.pct_change().iloc[1:]
    x = x.to_frame()
    x = x.values
    
    y = y.pct_change().iloc[1:]
    y = y.to_frame()
    y = y.values
    
    
    x = add_constant(x)
    model = OLS(y,x).fit()
    x = x[:, 1]
    
    return model.params[1]

In [2]:
from statsmodels.tsa.stattools import coint, adfuller

In [3]:
def drop_fun(data):
    columns_to_drop_us = list(pd.read_csv("all_del.csv").iloc[:, 0])
    data = data.drop(columns_to_drop_us, axis = 1, errors="ignore")
    return data

def drop_fun_corr(data):
    columns_to_drop_us = list(pd.read_csv("all_del.csv").iloc[:, 0])
    data = data.drop(columns_to_drop_us, axis = 1, errors="ignore")
    data = data.drop(columns_to_drop_us, axis = 0, errors="ignore")

    return data

In [4]:
us_df          = drop_fun(get_simstock_result("nasdaq_nasdaq_2022"))

In [5]:
def cointegration_checker(query, stock_dataframe):
    cointegrated_pairs = []
    
    k = stock_dataframe.shape[1]
    p_values = np.ones(k )
    
    keys = stock_dataframe.keys()
    #getting the column names (tickers) from df
    
    for i in range(k):            
        Asset_1 = stock_dataframe[query]
        Asset_2 = stock_dataframe[keys[i]]
        
        #iterating through the df and testing cointegration for all pairs of tickers
        Coint_Test = coint(Asset_1, Asset_2)
        
        pvalue = Coint_Test[1]
        p_values[i] = pvalue
        if pvalue < 0.02:
            cointegrated_pairs.append((pvalue,keys[i]))
            
    return p_values, cointegrated_pairs

In [6]:
target_data = yf.download(list(us_df.columns), start="2022-01-01", end="2023-01-01", progress=False)["Adj Close"] 


13 Failed downloads:
- CPE: No data found, symbol may be delisted
- ACOR: No data found, symbol may be delisted
- EGLE: No data found, symbol may be delisted
- NGM: No data found, symbol may be delisted
- CASA: No data found, symbol may be delisted
- PGTI: No data found, symbol may be delisted
- MTBL: No data found, symbol may be delisted
- DSKE: No data found, symbol may be delisted
- ONTX: No data found, symbol may be delisted
- SCTL: No data found, symbol may be delisted
- CCLP: No data found, symbol may be delisted
- CVCY: No data found, symbol may be delisted
- AMNB: No data found, symbol may be delisted


In [7]:
target_data = target_data.dropna(axis = 1)

In [8]:
aapl_data = cointegration_checker("AAPL", target_data)
cmg_data = cointegration_checker("CMG", target_data)
MSFT_data = cointegration_checker("MSFT", target_data)

V_data = cointegration_checker("V", target_data)
WFC_data = cointegration_checker("WFC", target_data)
XOM_data = cointegration_checker("XOM", target_data)
PFE_data = cointegration_checker("PFE", target_data)

AMZN_data = cointegration_checker("AMZN", target_data)
BA_data = cointegration_checker("BA", target_data)

META_data = cointegration_checker("META", target_data)
MA_data = cointegration_checker("MA", target_data)
CVS_data = cointegration_checker("CVS", target_data)

/opt/conda/envs/bigcon/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1721: CollinearityWarning: y0 and y1 are (almost) perfectly colinear.Cointegration test is not reliable in this case.
  warnings.warn(
/opt/conda/envs/bigcon/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1721: CollinearityWarning: y0 and y1 are (almost) perfectly colinear.Cointegration test is not reliable in this case.
  warnings.warn(
/opt/conda/envs/bigcon/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1721: CollinearityWarning: y0 and y1 are (almost) perfectly colinear.Cointegration test is not reliable in this case.
  warnings.warn(
/opt/conda/envs/bigcon/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1721: CollinearityWarning: y0 and y1 are (almost) perfectly colinear.Cointegration test is not reliable in this case.
  warnings.warn(
/opt/conda/envs/bigcon/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1721: CollinearityWarning: y0 and y1 are (almost) perfectly 

In [9]:
aapl_data = sorted(aapl_data[1], key=lambda x: x[0])[:4]
cmg_data = sorted(cmg_data[1], key=lambda x: x[0])[:4]
MSFT_data = sorted(MSFT_data[1], key=lambda x: x[0])[:4]
V_data = sorted(V_data[1], key=lambda x: x[0])[:4]
WFC_data = sorted(WFC_data[1], key=lambda x: x[0])[:4]
XOM_data = sorted(XOM_data[1], key=lambda x: x[0])[:4]
PFE_data = sorted(PFE_data[1], key=lambda x: x[0])[:4]
AMZN_data = sorted(AMZN_data[1], key=lambda x: x[0])[:4]
BA_data = sorted(BA_data[1], key=lambda x: x[0])[:4]
META_data = sorted(META_data[1], key=lambda x: x[0])[:4]
MA_data = sorted(MA_data[1], key=lambda x: x[0])[:4]
CVS_data = sorted(CVS_data[1], key=lambda x: x[0])[:4]

In [10]:
aapl_data_l = [x[1] for x in aapl_data]
cmg_data_l = [x[1] for x in cmg_data]
msft_data_l = [x[1] for x in MSFT_data]
v_data_l = [x[1] for x in V_data]
wfc_data_l = [x[1] for x in WFC_data]
xom_data_l = [x[1] for x in XOM_data]
pfe_data_l = [x[1] for x in PFE_data]
amzn_data_l = [x[1] for x in AMZN_data]
ba_data_l = [x[1] for x in BA_data]
meta_data_l = [x[1] for x in META_data]
ma_data_l = [x[1] for x in MA_data]
cvs_data_l = [x[1] for x in CVS_data]

In [11]:
aapl_main, main_stat_, _ = run_pair(aapl_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
cmg_main,  cmg_stat_, _ = run_pair(cmg_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
msft_main,  msft_stat_, _ = run_pair(msft_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
v_main,     v_stat_, _ = run_pair(v_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
wfc_main,  wfc_stat_, _ = run_pair(wfc_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
xom_main,  xom_stat_, _ = run_pair(xom_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
pfe_main,  pfe_stat_, _ = run_pair(pfe_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
amzn_main,  amzn_stat_, _ = run_pair(amzn_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
ba_main,   ba_stat_, _ = run_pair(ba_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
meta_main,  meta_stat_, _ = run_pair(meta_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
ma_main,   ma_stat_, _ = run_pair(ma_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)
cvs_main,  cvs_stat_, _ = run_pair(cvs_data_l, 1.25, "COINT", "2022-01-01", "2023-01-01", 500)

In [12]:
wealth_df = pd.concat([pd.DataFrame({0 }),
           pd.DataFrame(cmg_stat_[0]).iloc[2:, :],
           pd.DataFrame(msft_stat_[0]).iloc[2:, :],
           pd.DataFrame(wfc_stat_[0]).iloc[2:, :],
           pd.DataFrame(v_stat_[0]).iloc[2:, :],
           pd.DataFrame(xom_stat_[0]).iloc[2:, :],
           pd.DataFrame(pfe_stat_[0]).iloc[2:, :],
           pd.DataFrame(amzn_stat_[0]).iloc[2:, :],
           pd.DataFrame(ba_stat_[0]).iloc[2:, :],
           pd.DataFrame(meta_stat_[0]).iloc[2:, :],
           pd.DataFrame(ma_stat_[0]).iloc[2:, :],
           pd.DataFrame(cvs_stat_[0]).iloc[2:, :],
          ])

In [13]:
wealth_std = pd.concat([pd.DataFrame({0 }),
           pd.DataFrame(cmg_stat_[1]).iloc[2:, :],
           pd.DataFrame(msft_stat_[1]).iloc[2:, :],
           pd.DataFrame(wfc_stat_[1]).iloc[2:, :],
           pd.DataFrame(v_stat_[1]).iloc[2:, :],
           pd.DataFrame(xom_stat_[1]).iloc[2:, :],
           pd.DataFrame(pfe_stat_[1]).iloc[2:, :],
           pd.DataFrame(amzn_stat_[1]).iloc[2:, :],
           pd.DataFrame(ba_stat_[1]).iloc[2:, :],
           pd.DataFrame(meta_stat_[1]).iloc[2:, :],
           pd.DataFrame(ma_stat_[1]).iloc[2:, :],
           pd.DataFrame(cvs_stat_[1]).iloc[2:, :],
          ])

In [14]:
wealth_df = wealth_df.round(2).astype(str)
wealth_std = ' \tiny{\pm' + wealth_std.round(2).astype(str) +"}"

In [15]:
(wealth_df + wealth_std)

,0
0,0.0 \tiny{\pm0.0}
Wealth,-856.49 \tiny{\pm2964.75}
Wealth,454.68 \tiny{\pm857.63}
Wealth,-1309.26 \tiny{\pm2141.77}
Wealth,240.39 \tiny{\pm1067.06}
Wealth,1184.0 \tiny{\pm2526.09}
Wealth,419.17 \tiny{\pm210.61}
Wealth,-1183.58 \tiny{\pm3522.64}
Wealth,143.38 \tiny{\pm724.86}
Wealth,326.92 \tiny{\pm1266.86}


In [16]:
mdd_mean = pd.concat([pd.DataFrame({0 }),
           -pd.DataFrame(cmg_stat_[0]).iloc[0:1, :],
           -pd.DataFrame(msft_stat_[0]).iloc[0:1, :],
           -pd.DataFrame(wfc_stat_[0]).iloc[0:1, :],
           -pd.DataFrame(v_stat_[0]).iloc[0:1, :],
           -pd.DataFrame(xom_stat_[0]).iloc[0:1, :],
           -pd.DataFrame(pfe_stat_[0]).iloc[0:1, :],
           -pd.DataFrame(amzn_stat_[0]).iloc[0:1, :],
           -pd.DataFrame(ba_stat_[0]).iloc[0:1, :],
           -pd.DataFrame(meta_stat_[0]).iloc[0:1, :],
           -pd.DataFrame(ma_stat_[0]).iloc[0:1, :],
           -pd.DataFrame(cvs_stat_[0]).iloc[0:1, :],
          ])

In [17]:
mdd_std = pd.concat([pd.DataFrame({0 }),
           pd.DataFrame(cmg_stat_[1]).iloc[0:1, :],
           pd.DataFrame(msft_stat_[1]).iloc[0:1, :],
           pd.DataFrame(wfc_stat_[1]).iloc[0:1, :],
           pd.DataFrame(v_stat_[1]).iloc[0:1, :],
           pd.DataFrame(xom_stat_[1]).iloc[0:1, :],
           pd.DataFrame(pfe_stat_[1]).iloc[0:1, :],
           pd.DataFrame(amzn_stat_[1]).iloc[0:1, :],
           pd.DataFrame(ba_stat_[1]).iloc[0:1, :],
           pd.DataFrame(meta_stat_[1]).iloc[0:1, :],
           pd.DataFrame(ma_stat_[1]).iloc[0:1, :],
           pd.DataFrame(cvs_stat_[1]).iloc[0:1, :],
          ])

In [18]:
mdd_mean = (mdd_mean.round(2)).astype(str)
mdd_std = ' \tiny{\pm' + mdd_std.round(2).astype(str) +"}"

In [19]:
(mdd_mean + mdd_std)

,0
0,0.0 \tiny{\pm0.0}
Maximum Drawdown (%),-20.03 \tiny{\pm15.47}
Maximum Drawdown (%),-13.23 \tiny{\pm2.96}
Maximum Drawdown (%),-23.95 \tiny{\pm13.75}
Maximum Drawdown (%),-7.46 \tiny{\pm5.87}
Maximum Drawdown (%),-9.29 \tiny{\pm7.58}
Maximum Drawdown (%),-9.03 \tiny{\pm5.56}
Maximum Drawdown (%),-31.9 \tiny{\pm13.12}
Maximum Drawdown (%),-14.43 \tiny{\pm7.32}
Maximum Drawdown (%),-25.4 \tiny{\pm11.24}


In [20]:
wealth_df = pd.concat([pd.DataFrame({0 }),
           pd.DataFrame(cmg_main.iloc[2:3, :1].values[0]),
           pd.DataFrame(msft_main.iloc[2:3, :1].values[0]),
           pd.DataFrame(wfc_main.iloc[2:3, :1].values[0]),
           pd.DataFrame(v_main.iloc[2:3, :1].values[0]),
           pd.DataFrame(xom_main.iloc[2:3, :1].values[0]),
           pd.DataFrame(pfe_main.iloc[2:3, :1].values[0]),
           pd.DataFrame(amzn_main.iloc[2:3, :1].values[0]),
           pd.DataFrame(ba_main.iloc[2:3, :1].values[0]),
           pd.DataFrame(meta_main.iloc[2:3, :1].values[0]),
           pd.DataFrame(ma_main.iloc[2:3, :1].values[0]),
           pd.DataFrame(cvs_main.iloc[2:3, :1].values[0]),
          ])

In [21]:
mdd_df = pd.concat([pd.DataFrame({0 }),
           pd.DataFrame(cmg_main.iloc[0:1, :1].values[0]),
           pd.DataFrame(msft_main.iloc[0:1, :1].values[0]),
           pd.DataFrame(wfc_main.iloc[0:1, :1].values[0]),
           pd.DataFrame(v_main.iloc[0:1, :1].values[0]),
           pd.DataFrame(xom_main.iloc[0:1, :1].values[0]),
           pd.DataFrame(pfe_main.iloc[0:1, :1].values[0]),
           pd.DataFrame(amzn_main.iloc[0:1, :1].values[0]),
           pd.DataFrame(ba_main.iloc[0:1, :1].values[0]),
           pd.DataFrame(meta_main.iloc[0:1, :1].values[0]),
           pd.DataFrame(ma_main.iloc[0:1, :1].values[0]),
           pd.DataFrame(cvs_main.iloc[0:1, :1].values[0]),
          ])

In [22]:
wealth_df

,0
0,0.000
0,-59.034
0,1209.715
0,-2725.642
0,691.545
0,1102.321
0,442.029
0,2622.295
0,93.457
0,1788.357


In [23]:
-mdd_df

,0
0,-0.000
0,-8.637
0,-10.761
0,-31.518
0,-2.775
0,-1.398
0,-7.020
0,-27.161
0,-22.778
0,-26.082
